## Import Library and dataset

In [116]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split

In [117]:
df = pd.read_csv("../data/interests_without_onehot.csv")

features = df.iloc[:, :-1].values
labels = df["y"].values

dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [118]:
for features, label in dataset.take(31):
    print("features :", features)
    print("Label:", label)
    print("Label dtype:", label.dtype)

features : tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 0 2], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([2 1 2 0 0 0 1 2 2 0 1 2 0 0 1], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([0 2 0 0 0 1 2 2 0 0 0 1 1 0 0], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 1 1 1 1 1 1 1 1 0 1 2 1 0 1], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([2 2 0 1 1 1 2 2 1 0 2 0 2 0 2], shape=(15,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
feat

## Preprocessing Dataset

In [119]:
keys = tf.constant(["IT sector", "Goverment sector", "Health sector", "Education sector", "Sports sector", "Finance sector", "Entertainment sector"])
values = tf.constant([0, 1, 2, 3, 4, 5, 6])

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys, values),
    default_value=-1
)

def preprocess(features, label):
    label_int = table.lookup(label)
    label_encoded = tf.one_hot(label_int, depth=7)
    
    return features, label_encoded

dataset = dataset.map(preprocess)

In [120]:
for features, labels in dataset.take(31):
    print("Original Features:", features)
    print("One-Hot Encoded Labels:", labels.numpy())

Original Features: tf.Tensor([0 0 0 0 0 0 0 0 0 0 0 0 0 0 0], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 0 2], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([2 1 2 0 0 0 1 2 2 0 1 2 0 0 1], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([0 2 0 0 0 1 2 2 0 0 0 1 1 0 0], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 1 1 1 1 1 1 1 1 0 1 2 1 0 1], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([2 2 0 1 1 1 2 2 1 0 2 0 2 0 2], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 0 2 1 1 1 0 1 2 0 0 1 0 0 0], shape=(15,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([0 2 1 0 0 2 2 2 0 0 0 2 2

## Splitting Dataset

In [121]:
features = []
labels = []

for feature, label in dataset:
    features.append(feature.numpy())
    labels.append(label.numpy())

features = np.array(features)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42, stratify=labels
)


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [122]:
for features, labels in train_dataset.take(1):
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

Features shape: (32, 15)
Labels shape: (32, 7)


## Creating Model

In [123]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(15,)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Training Model

In [124]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=100,
)

Epoch 1/100


5/5 [==============================] - 1s 58ms/step - loss: 2.0563 - accuracy: 0.1698 - val_loss: 1.9013 - val_accuracy: 0.1750
Epoch 2/100
5/5 [==============================] - 0s 5ms/step - loss: 1.9081 - accuracy: 0.2390 - val_loss: 1.8426 - val_accuracy: 0.3000
Epoch 3/100
5/5 [==============================] - 0s 6ms/step - loss: 1.8412 - accuracy: 0.2201 - val_loss: 1.7882 - val_accuracy: 0.3500
Epoch 4/100
5/5 [==============================] - 0s 7ms/step - loss: 1.7843 - accuracy: 0.2642 - val_loss: 1.7335 - val_accuracy: 0.4250
Epoch 5/100
5/5 [==============================] - 0s 6ms/step - loss: 1.6943 - accuracy: 0.3459 - val_loss: 1.6753 - val_accuracy: 0.4500
Epoch 6/100
5/5 [==============================] - 0s 6ms/step - loss: 1.6605 - accuracy: 0.3774 - val_loss: 1.6161 - val_accuracy: 0.4250
Epoch 7/100
5/5 [==============================] - 0s 6ms/step - loss: 1.6857 - accuracy: 0.2830 - val_loss: 1.5559 - val_accuracy: 0.5250
Epoch 8/100
5/5 [=====================